In [ ]:
#default_exp training

In [ ]:
#export
import numpy as np
import pandas as pd
import os
from deeplearning_image_classification import data_loading

from sklearn import model_selection, metrics
import tensorflow as tf
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)
tf.keras.backend.set_floatx('float16')
tf.keras.backend.set_epsilon(1e-4)
from tensorflow import keras
from tensorflow.keras.mixed_precision import experimental as mixed_precision
import keras_applications
#export


np.random.seed(0)
pd.set_option('display.max_colwidth', 100)
DATA_DIR = data_loading.DATA_DIR

In [ ]:
%cd ..

/home/kuba/Projects/deeplearning_image_classification


Try removing this line on your PC, maybe you will not encounter [this issue](https://github.com/tensorflow/tensorflow/issues/24496)

In [ ]:
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_policy(policy)

In [ ]:
#export
train_csv_path = os.path.join(DATA_DIR, 'train_metadata.csv')
test_csv_path = os.path.join(DATA_DIR, 'test_metadata.csv')

In [ ]:
#export
def get_metadata(train_metadata_path=train_csv_path, test_metadata_path=test_csv_path):
    train_metadata_df = pd.read_csv(train_csv_path)
    test_metadata_df = pd.read_csv(test_csv_path)
    return train_metadata_df, test_metadata_df

In [ ]:
train_csv_path

'data/train_metadata.csv'

In [ ]:
train_metadata_df, test_metadata_df = get_metadata()

In [ ]:
sample_size = 11000
sample_val_size = 1000
__, sample_train_val_metadata_df = model_selection.train_test_split(train_metadata_df, test_size=sample_size, random_state=2, stratify=train_metadata_df['class'])
sample_train_metadata_df, sample_val_metadata_df = model_selection.train_test_split(sample_train_val_metadata_df, test_size=sample_val_size, random_state=2, stratify=sample_train_val_metadata_df['class'])

In [ ]:
#export
image_size = (224, 224)
train_batch_size = 64 
val_batch_size = 64
test_batch_size = 64


def get_train_val_test_iterators(train_metadata_df, val_metadata_df, test_metadata_df, image_size, train_batch_size, val_batch_size, test_batch_size):
    image_gen = keras.preprocessing.image.ImageDataGenerator(
        preprocessing_function=keras.applications.mobilenet.preprocess_input,
        horizontal_flip=True
    )

    train_image_iterator = image_gen.flow_from_dataframe(train_metadata_df, batch_size=train_batch_size, target_size=image_size,
        shuffle=True)
    val_image_iterator = image_gen.flow_from_dataframe(val_metadata_df, batch_size=val_batch_size, target_size=image_size, shuffle=False)
    test_image_iterator = image_gen.flow_from_dataframe(test_metadata_df, batch_size=test_batch_size, target_size=image_size, shuffle=False)
    return train_image_iterator, val_image_iterator, test_image_iterator

In [ ]:
train_image_iterator, val_image_iterator, test_image_iterator = get_train_val_test_iterators(sample_train_metadata_df, sample_val_metadata_df, test_metadata_df, image_size=image_size, train_batch_size=train_batch_size, test_batch_size=test_batch_size, val_batch_size=val_batch_size) 
n_classes = len(train_image_iterator.class_indices)

Found 10000 validated image filenames belonging to 82 classes.
Found 1000 validated image filenames belonging to 82 classes.
Found 10000 validated image filenames belonging to 82 classes.


# Setting up model

We use pretrained MobileNet model for feature extraction

In [ ]:
#export
learning_rate = 0.001
beta_1 = 0.9
beta_2 = 0.999
optimizer = keras.optimizers.Adam(learning_rate=learning_rate, beta_1=beta_1, beta_2=beta_2)
default_metrics = ['acc', keras.metrics.Precision(), keras.metrics.Recall()]
pretrained = False
freeze_pretrained = False
last_layer_convolutions = 64


def setup_model(n_classes, optimizer, metrics):
    weights =  'imagenet' if pretrained else None
    base_model = keras_applications.mobilenet.MobileNet(weights=None, input_shape=(224, 224, 3), backend=tf.keras.backend, layers=tf.keras.layers, models=tf.keras.models, utils=tf.keras.utils) 
    if freeze_pretrained:
        base_model.trainable = False
    model = keras.Sequential(
        [
            base_model,
            keras.layers.Flatten(),
            keras.layers.Dense(n_classes, dtype='float16'),
            keras.layers.Softmax()
        ])
    model.compile(
        loss=keras.losses.CategoricalCrossentropy(),
        optimizer=optimizer,
        metrics=metrics)
    return model

In [ ]:
model = setup_model(n_classes, optimizer, default_metrics)

In [ ]:
model.fit(train_image_iterator, validation_data=val_image_iterator, epochs=10, use_multiprocessing=True, workers=5)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 157 steps, validate for 16 steps
Epoch 1/10
157/157 [==============================] - 71s 451ms/step - loss: 4.0748 - acc: 0.0824 - precision_1: 0.0435 - recall_1: 5.0000e-04 - val_loss: 3.9044 - val_acc: 0.0780 - val_precision_1: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 2/10
157/157 [==============================] - 60s 383ms/step - loss: 4.0086 - acc: 0.1015 - precision_1: 0.0824 - recall_1: 0.0015 - val_loss: 3.9302 - val_acc: 0.0970 - val_precision_1: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 3/10
157/157 [==============================] - 61s 388ms/step - loss: 3.9870 - acc: 0.1113 - precision_1: 0.1364 - recall_1: 0.0033 - val_loss: 3.9795 - val_acc: 0.0780 - val_precision_1: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 4/10
157/157 [==============================] - 66s 422ms/step - loss: 3.8598 - acc: 0.1248 - precision_1: 0.3929 - recall_1: 0.0044 - val_loss: 4.0002 - val_acc: 0.0710 - val_precision_1: 0.0000e+0

In [ ]:
model.evaluate(test_image_iterator)

  ...
    to  
  ['...']
157/157 [==============================] - 41s 262ms/step - loss: 3.7567 - acc: 0.1340 - precision_1: 0.7345 - recall_1: 0.0083


[3.7567426353503186, 0.134, 0.7345133, 0.0083]

In [ ]:
y_test_pred = model.predict(test_image_iterator).argmax(axis=1)

In [ ]:
y_test = np.array([test_image_iterator.class_indices[c] for c in test_metadata_df['class']])

## Test set accuracy

In [ ]:
metrics.accuracy_score(y_test, y_test_pred)

0.1321

## Script mode

In [ ]:
#export
sample_size = 11000
val_size = 1000
epochs = 20


if __name__ == '__main__':
    train_val_metadata_df, test_metadata_df = get_metadata()
    
    if sample_size != 'all':
        __, train_val_metadata_df = model_selection.train_test_split(
            train_val_metadata_df,
            test_size=sample_size,
            random_state=2,
            stratify=train_val_metadata_df['class']
        )
    train_metadata_df, val_metadata_df = model_selection.train_test_split(
        train_val_metadata_df,
        test_size=val_size,
        random_state=2,
        stratify=train_val_metadata_df['class']
    )
    
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir='logs', histogram_freq=0, write_graph=True, write_images=False,
        update_freq='epoch', profile_batch=2, embeddings_freq=0,
        embeddings_metadata=None
    )

    train_image_iterator, val_image_iterator, test_image_iterator = get_train_val_test_iterators(
        train_metadata_df, val_metadata_df, test_metadata_df,
        image_size=image_size, train_batch_size=train_batch_size, test_batch_size=test_batch_size, val_batch_size=val_batch_size) 
    n_classes = len(train_image_iterator.class_indices)
    model = setup_model(n_classes, optimizer, default_metrics)
    model.fit(train_image_iterator, validation_data=val_image_iterator, epochs=10, callbacks=[tensorboard_callback])
    model.save('data/model.hdf5')